#### Pandas处理分析网站原始访问日志
目标：真实项目的实战，探索Pandas的数据处理与分析

实例：kaggle下载 .log 文件

实现步骤：
1. 读取数据，清理，格式化
2. 统计爬虫spider的访问比例，输出柱状图
3. 统计http状态码的访问占比，输出饼图
4. 统计按小时，按天的PV/UV流量趋势，输出折线图

#### 1. 读取数据并清理格式化

In [27]:
import pandas as pd
import numpy as np
# pandas对列中显示的字符数有一些限制，默认值为50字符。所以，有的值字符过长就会显示省略号。如果想全部显示，可以设置display.max_colwidth
pd.set_option('display.max_colwidth',-1)

from pyecharts import options as opts
from pyecharts.charts import Bar,Pie,Line

C:\Users\pxpxz_ct9p1p3\AppData\Local\Temp\ipykernel_12752\1923315685.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth',-1)


In [28]:
# 读取数据
df=pd.read_csv('./files/access.log',sep=' ',header=None, error_bad_lines=False)
df.head()

C:\Users\pxpxz_ct9p1p3\AppData\Local\Temp\ipykernel_12752\2832735794.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('./files/access.log',sep=' ',header=None, error_bad_lines=False)


,0,1,2,3,4,5,6,7,8,9,10
0,54.36.149.41,-,-,[22/Jan/2019:03:56:14,+0330],"GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1",200,30577,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/),-
1,31.56.96.51,-,-,[22/Jan/2019:03:56:16,+0330],GET /image/60844/productModel/200x200 HTTP/1.1,200,5667,https://www.zanbil.ir/m/filter/b113,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36",-
2,31.56.96.51,-,-,[22/Jan/2019:03:56:16,+0330],GET /image/61474/productModel/200x200 HTTP/1.1,200,5379,https://www.zanbil.ir/m/filter/b113,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36",-
3,40.77.167.129,-,-,[22/Jan/2019:03:56:17,+0330],GET /image/14925/productModel/100x100 HTTP/1.1,200,1696,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm),-
4,91.99.72.15,-,-,[22/Jan/2019:03:56:17,+0330],GET /product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%D8%B1-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C-%D9%BE%D8%B1%D9%86%D8%B3%D9%84%DB%8C-%D9%85%D8%AF%D9%84-PR257AT HTTP/1.1,200,41483,-,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16.0)Gecko/16.0 Firefox/16.0,-


In [29]:
# 选取要用的列
df=df[[0,3,6,9]].copy()
df.head()

,0,3,6,9
0,54.36.149.41,[22/Jan/2019:03:56:14,200,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)
1,31.56.96.51,[22/Jan/2019:03:56:16,200,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36"
2,31.56.96.51,[22/Jan/2019:03:56:16,200,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36"
3,40.77.167.129,[22/Jan/2019:03:56:17,200,Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)
4,91.99.72.15,[22/Jan/2019:03:56:17,200,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16.0)Gecko/16.0 Firefox/16.0


In [30]:
# 重新给columns命名
df.columns=['ip','stime','status','client']
df.head()

,ip,stime,status,client
0,54.36.149.41,[22/Jan/2019:03:56:14,200,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)
1,31.56.96.51,[22/Jan/2019:03:56:16,200,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36"
2,31.56.96.51,[22/Jan/2019:03:56:16,200,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36"
3,40.77.167.129,[22/Jan/2019:03:56:17,200,Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)
4,91.99.72.15,[22/Jan/2019:03:56:17,200,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16.0)Gecko/16.0 Firefox/16.0


In [31]:
df.dtypes

ip        object
stime     object
status    int64 
client    object
dtype: object

#### 2. 统计spider的比例

In [32]:
df['is_spider']=df['client'].str.lower().str.contains('spider')

In [33]:
df.head()

,ip,stime,status,client,is_spider
0,54.36.149.41,[22/Jan/2019:03:56:14,200,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/),False
1,31.56.96.51,[22/Jan/2019:03:56:16,200,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36",False
2,31.56.96.51,[22/Jan/2019:03:56:16,200,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36",False
3,40.77.167.129,[22/Jan/2019:03:56:17,200,Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm),False
4,91.99.72.15,[22/Jan/2019:03:56:17,200,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16.0)Gecko/16.0 Firefox/16.0,False


In [34]:
df_spider=df['is_spider'].value_counts()
df_spider

False    10364523
True     622     
Name: is_spider, dtype: int64

In [35]:
bar=(
    Bar().add_xaxis([str(x) for x in df_spider.index])
         .add_yaxis('是否Spider', df_spider.values.tolist())
         .set_global_opts(title_opts=opts.TitleOpts(title='爬虫访问量占比'))
)
bar.render_notebook()

#### 3. 访问状态码的数量对比

In [36]:
df_status=df.groupby('status').size() # 和 df_status=df['status'].value_counts()效果一样
df_status

status
200    9579825
206    3      
301    67553  
302    199835 
304    340228 
400    586    
401    323    
403    5634   
404    105011 
405    6      
408    112    
414    17     
499    50852  
500    14266  
502    798    
504    103    
dtype: int64

In [37]:
list(zip(df_status.index, df_status))

[(200, 9579825),
 (206, 3),
 (301, 67553),
 (302, 199835),
 (304, 340228),
 (400, 586),
 (401, 323),
 (403, 5634),
 (404, 105011),
 (405, 6),
 (408, 112),
 (414, 17),
 (499, 50852),
 (500, 14266),
 (502, 798),
 (504, 103)]

In [38]:
pie=(
    Pie()
    .add('状态码比例',list(zip(df_status.index, df_status)))
    .set_series_opts(label_opts=opts.LabelOpts(formatter='{b}:{c}'))
)
pie.render_notebook()
#### 奇怪！！！为什么没有图列在饼图上方（应该有，可供选择）

#### 4. 实现按小时，按天粒度的流量统计

In [39]:
df.head()

,ip,stime,status,client,is_spider
0,54.36.149.41,[22/Jan/2019:03:56:14,200,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/),False
1,31.56.96.51,[22/Jan/2019:03:56:16,200,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36",False
2,31.56.96.51,[22/Jan/2019:03:56:16,200,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36",False
3,40.77.167.129,[22/Jan/2019:03:56:17,200,Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm),False
4,91.99.72.15,[22/Jan/2019:03:56:17,200,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16.0)Gecko/16.0 Firefox/16.0,False


In [40]:
df['stime']=pd.to_datetime(df['stime'].str[1:],format='%d/%b/%Y:%H:%M:%S')
# tips: .str[:1] 表示去掉中括号

df.head()

,ip,stime,status,client,is_spider
0,54.36.149.41,2019-01-22 03:56:14,200,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/),False
1,31.56.96.51,2019-01-22 03:56:16,200,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36",False
2,31.56.96.51,2019-01-22 03:56:16,200,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36",False
3,40.77.167.129,2019-01-22 03:56:17,200,Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm),False
4,91.99.72.15,2019-01-22 03:56:17,200,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16.0)Gecko/16.0 Firefox/16.0,False


In [41]:
df.set_index('stime',inplace=True)
df.sort_index(inplace=True)
df.head()

,ip,status,client,is_spider
stime,,,,
2019-01-22 03:56:14,54.36.149.41,200,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/),False
2019-01-22 03:56:16,31.56.96.51,200,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36",False
2019-01-22 03:56:16,31.56.96.51,200,"Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36",False
2019-01-22 03:56:17,40.77.167.129,200,Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm),False
2019-01-22 03:56:17,91.99.72.15,200,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16.0)Gecko/16.0 Firefox/16.0,False


In [42]:
df.index

DatetimeIndex(['2019-01-22 03:56:14', '2019-01-22 03:56:16',
               '2019-01-22 03:56:16', '2019-01-22 03:56:17',
               '2019-01-22 03:56:17', '2019-01-22 03:56:17',
               '2019-01-22 03:56:18', '2019-01-22 03:56:18',
               '2019-01-22 03:56:18', '2019-01-22 03:56:18',
               ...
               '2019-01-26 20:29:13', '2019-01-26 20:29:13',
               '2019-01-26 20:29:13', '2019-01-26 20:29:13',
               '2019-01-26 20:29:13', '2019-01-26 20:29:13',
               '2019-01-26 20:29:13', '2019-01-26 20:29:13',
               '2019-01-26 20:29:13', '2019-01-26 20:29:13'],
              dtype='datetime64[ns]', name='stime', length=10365152, freq=None)

In [46]:
# 按小时统计 (pv:计数，uv:去重计数)
df_pvuv=df.resample('H')['ip'].agg(pv=np.size, uv=pd.Series.nunique)

# 按每6个小时统计
#df_pvuv=df.resample('6H')['ip'].agg(pv=np.size, uv=pd.Series.nunique)

# 按天统计
#df_pvuv=df.resample('D')['ip'].agg(pv=np.size, uv=pd.Series.nunique)

df_pvuv.head()

,pv,uv
stime,,
2019-01-22 03:00:00,1285,116
2019-01-22 04:00:00,14471,767
2019-01-22 05:00:00,14867,762
2019-01-22 06:00:00,19357,1263
2019-01-22 07:00:00,47800,2149


In [47]:
line=(
    Line()
    .add_xaxis(df_pvuv.index.to_list())
    .add_yaxis('pv', df_pvuv['pv'].to_list())
    .add_yaxis('uv', df_pvuv['uv'].to_list())
    .set_global_opts(
            title_opts=opts.TitleOpts(title='PVUV数据对比'),
            tooltip_opts=opts.TooltipOpts(trigger='axis',axis_pointer_type='cross')  
    )
)
line.render_notebook()